In [3]:
# Import libraries
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
from functools import reduce
from pathlib import Path

# List all data files which should be preprocessed
directory = Path('/Users/anja/Desktop/data_mt')
data_files = [file.name for file in directory.glob('*.csv')]

# Columns needed from raw file
desired_columns = ['ident_block_trial', # identifier to determine trial, because dataset is off
                    'participant',
                    'cc_owner.started', # time when the owner confirm dialog in cc trials was started (to measure if something was off with keep)
                    'cc_owner.stopped',
                    'yc_owner.started', # time when the owner confirm dialog in yc trials was started (to measure if something was off with keep)
                    'yc_owner.stopped',
                    'balance.started',
                    'balance.stopped',
                    'fixation.started',
                    'fixation.stopped',
                    'round_info.started',
                    'round_info.stopped',
                    'yc_choice.started',
                    'yc_choice.stopped',
                    'cc_choice.started',
                    'cc_choice.stopped',
                    'cc_choice_confirm.started',
                    'cc_choice_confirm.stopped',
                    'yc_draw_frame.started',
                    'yc_draw_frame.stopped',
                    'yc_value.started',
                    'yc_value.stopped',
                    'yc_owner_confirm.started',
                    'yc_owner_confirm.stopped',
                    'cc_value.started',
                    'cc_value.stopped',
                    'cc_owner_confirm.started',
                    'cc_owner_confirm.stopped'
                    ] 

# Create empty list to store files
dfs = []
# Read the csv files and create columns in case some are missing
for file_name in data_files:
    file_path = os.path.join(directory, file_name)
    # Get the available columns
    df_initial = pd.read_csv(file_path, nrows=0) 
    available_columns = df_initial.columns.tolist()
    # Determine which desired columns are available in csv
    existing_columns = [col for col in desired_columns if col in available_columns]
    # Read the CSV file, using only the existing columns
    df = pd.read_csv(file_path, usecols=existing_columns)
    # Add the missing columns with empty values
    for col in desired_columns:
        if col not in df.columns:
            df[col] = pd.NA

    dfs.append(df)

    # Concatenating files together
    concatenated_df = pd.concat(dfs, ignore_index=True)

/var/folders/94/kbz524cd32j80_2tm_hwnr780000gn/T/ipykernel_11466/2933139044.py:64: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  concatenated_df = pd.concat(dfs, ignore_index=True)
/var/folders/94/kbz524cd32j80_2tm_hwnr780000gn/T/ipykernel_11466/2933139044.py:64: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  concatenated_df = pd.concat(dfs, ignore_index=True)
/var/folders/94/kbz524cd32j80_2tm_hwnr780000gn/T/ipykernel_11466/2933139044.py:64: FutureWarning: The behavior of DataFrame concatenation 

KeyboardInterrupt: 

In [ ]:

# Putting owner times in one column
concatenated_df['owner_started'] = concatenated_df['cc_owner.started'].combine_first(concatenated_df['yc_owner.started'])
concatenated_df['owner_stopped'] = concatenated_df['cc_owner.stopped'].combine_first(concatenated_df['yc_owner.stopped'])
# Putting value times in one column
concatenated_df['value_started'] = concatenated_df['cc_value.started'].combine_first(concatenated_df['yc_value.started'])
concatenated_df['value_stopped'] = concatenated_df['cc_value.stopped'].combine_first(concatenated_df['yc_value.stopped'])
# Putting owner confirm times in one column
concatenated_df['owner_confirm_started'] = concatenated_df['cc_owner_confirm.started'].combine_first(concatenated_df['yc_owner_confirm.started'])
concatenated_df['owner_confirm_stopped'] = concatenated_df['cc_owner_confirm.stopped'].combine_first(concatenated_df['yc_owner_confirm.stopped'])
# Putting choice confirm times in one column
concatenated_df['choice_confirm_started'] = concatenated_df['cc_choice_confirm.started'].combine_first(concatenated_df['yc_draw_frame.started'])
concatenated_df['choice_confirm_stopped'] = concatenated_df['cc_choice_confirm.stopped'].combine_first(concatenated_df['yc_draw_frame.stopped'])

# Calculate durations
#concatenated_df['round_info_duration'] = concatenated_df['round_info.stopped'] - concatenated_df['round_info.started']
concatenated_df['fixation_duration'] = concatenated_df['fixation.stopped'] - concatenated_df['fixation.started']
concatenated_df['balance_duration'] = concatenated_df['balance.stopped'] - concatenated_df['balance.started']

concatenated_df['cc_choice_duration'] = concatenated_df['cc_choice.stopped'] - concatenated_df['cc_choice.started']
concatenated_df['yc_choice_duration'] = concatenated_df['yc_choice.stopped'] - concatenated_df['yc_choice.started']

concatenated_df['owner_confirm_duration'] = concatenated_df['owner_confirm_stopped'] - concatenated_df['owner_confirm_started']
concatenated_df['value_duration'] = concatenated_df['value_stopped'] - concatenated_df['value_started']
concatenated_df['owner_duration'] = concatenated_df['owner_stopped'] - concatenated_df['owner_started']
concatenated_df['choice_confirm_duration'] = concatenated_df['choice_confirm_stopped'] - concatenated_df['choice_confirm_started']
